## 1st Level of Transformation (Bronze to Silver)

In [0]:
dbutils.fs.ls('mnt/bronze/Sales/')

[FileInfo(path='dbfs:/mnt/bronze/Sales/CountryRegionCurrency/', name='CountryRegionCurrency/', size=0, modificationTime=1710923531000),
 FileInfo(path='dbfs:/mnt/bronze/Sales/CreditCard/', name='CreditCard/', size=0, modificationTime=1710923527000),
 FileInfo(path='dbfs:/mnt/bronze/Sales/Currency/', name='Currency/', size=0, modificationTime=1710923530000),
 FileInfo(path='dbfs:/mnt/bronze/Sales/CurrencyRate/', name='CurrencyRate/', size=0, modificationTime=1710923513000),
 FileInfo(path='dbfs:/mnt/bronze/Sales/Customer/', name='Customer/', size=0, modificationTime=1710923531000),
 FileInfo(path='dbfs:/mnt/bronze/Sales/PersonCreditCard/', name='PersonCreditCard/', size=0, modificationTime=1710923512000),
 FileInfo(path='dbfs:/mnt/bronze/Sales/SalesOrderDetail/', name='SalesOrderDetail/', size=0, modificationTime=1710923520000),
 FileInfo(path='dbfs:/mnt/bronze/Sales/SalesOrderHeader/', name='SalesOrderHeader/', size=0, modificationTime=1710923528000),
 FileInfo(path='dbfs:/mnt/bronze/S

In [0]:
dbutils.fs.ls('mnt/silver/')   

[]

In [0]:
table_name =[]

for i in dbutils.fs.ls('mnt/bronze/Sales/'):
    table_name.append(i.name.split('/')[0])

In [0]:
table_name

In [0]:
from pyspark.sql.functions import from_utc_timestamp, date_format
from pyspark.sql.types import TimestampType

for i in table_name:
    path = '/mnt/bronze/Sales/'+i+'/'+i+'.parquet'
    df=spark.read.format('parquet').load(path)
    column = df.columns

    for col in column:
        if 'Date' in col or 'date' in col:
            df = df.withColumn(col, date_format(from_utc_timestamp(df[col].cast(TimestampType()),"UTC"),'yyyy-MM-dd'))

    output_path = '/mnt/silver/Sales/'+i+'/'
    df.write.format('delta').mode('overwrite').save(output_path)